<a href="https://colab.research.google.com/github/vikpy/AISem3/blob/master/Test/M1_Test_AI_Packman_A_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Question - https://www.hackerrank.com/challenges/pacman-astar

 

take input as position of food and starting state on the pac man grid

 

Submission by end of day today

 

Sanity : Code Should be modular

 

Grading Pattern :

Code for the logic == 5 Marks

Code Explanation through comments == 2 Marks

Printing Shortest Path  == 2 Marks

Class - Object Usage == 1 Marks

Ans: 
We will use [heapq](https://towardsdatascience.com/priority-queues-in-python-3baf0bac2097) structure in python for implementation of the algorithm for popping values with least cost


[__Pseudo Code for A*:__](https://brilliant.org/wiki/a-star-search/)


```
make an openlist containing only the starting node

make an empty closed list

# f = h + g
while (the destination node has not been reached):
       consider the node with the lowest f score in the open list
       if (this node is our destination node) :
           we are finished 
       if not:
           put the current node in the closed list and look at all of its neighbors
           for (each neighbor of the current node):
               if (neighbor has lower g value than current and is in the closed list) :
                   replace the neighbor with the new, lower, g value 
                   current node is now the neighbor's parent            
               else if (current g value is lower and this neighbor is in the open list ) :
                   replace the neighbor with the new, lower, g value 
                   change the neighbor's parent to our current node

               else if this neighbor is not in both lists:
                   add it to the open list and set its g

```


In [16]:
# Defining Priority Queue in Python that will be used for assigning priorty based on f (= g + h) score value 

# Importing python pq implementation which returns the min element 

import heapq


class PriorityQueue:
  def __init__(self):
    self._queue = []
    self._index = 0

  def is_empty(self):
    return not self._queue

  def push(self, item, priority):
    heapq.heappush(self._queue, (priority, self._index, item))
    self._index += 1   

  def pop(self):
    return heapq.heappop(self._queue)[-1]

          
  def contains(self, item):
    return not self.is_empty() and item in list(zip(*self._queue))[1]


# Testint the priority queue

pq = PriorityQueue()
# Adding values in random order
pq.push('A',1)
pq.push('B',2)
pq.push('E',5)
pq.push('C',3)
pq.push('D',4)

if pq.pop() == 'A':
  print("Queue is working")
else : 
  print("Check the implementation")

Queue is working


In [9]:
pq.is_empty()

False

In [10]:
pq.pop()

'B'

In [18]:
# Creating the Pacman class for traversing from start to end 
class Pacman:
  
  # Input that is given as the input on hackerrank problem for the exam
  def __init__(self, start_row, start_col, food_row, food_col, grid):
    self.__start_row =  start_row
    self.__start_col = start_col
    self.__food_row = food_row
    self.__food_col = food_col
    self.__grid = grid

  # Defing the heuristic with choice of choosing between mahattan or euclidean 
  def __heuristic(self, source_x, source_y,  dest_x, dest_y, type):
    if type =="manhattan":
      dist = abs(dest_x - source_x) + abs(dest_y - source_y)
    elif type == "euclidean":
      dist =  ((dest_x - source_x)**2 + (dest_y - source_y)**2)**(0.5)
    return dist

  # Finding out whether the goal node is reached or not 
  def __is_goal(self, pos_x, pos_y, grid):
    return grid[pos_x][pos_y]  == '.'

  # Finding out whether the goal node is reached or not 
  def __is_wall(self, pos_x, pos_y, grid):
    return grid[pos_x][pos_y]  == '%'    
  
  # Finding the start position based on the problem 
  def __is_start(self, pos_x, pos_y, grid):
    return grid[pos_x][pos_y]  == 'P'
  
  # Finding the cost(g) every move
  def __cost(self, pos_x, pos_y, grid):
    cost = 0 
    if self.__is_goal(pos_x, pos_y, grid):
      cost = 1
    return cost

  # Feasible next positions 
  def __next_pos(self, pos_x, pos_y, grid):
    all_poss_next_positions = [\
                               (pos_x + 1, pos_y ), \
                               (pos_x , pos_y + 1 ),\
                               (pos_x , pos_y - 1  ),\
                               (pos_x - 1 , pos_y )\
                               ]
    next_pos = []                           
    for val in all_poss_next_positions:
      if not self.__is_wall(val[0], val[1], grid):
        next_pos.append(val)
    return next_pos 

  # Implementing the search algo from the start to the end   
  def __astar(self):
    pq = PriorityQueue()
    source  = {}
    cumumlative_cost = {}    
    goal_x = self.__food_row
    goal_y = self.__food_col
    start_x = self.__start_row
    start_y = self.__start_col
    grid = self.__grid
    # Starting from the first node 
    source[(start_x, start_y)] = None
    pq.push((start_x, start_y), 0)
    cumumlative_cost[(start_x, start_y)] = 0
    
    while not pq.is_empty():
      
      node = pq.pop()

      if self.__is_goal(node[0], node[1], grid):
        return source 
      
      # get the feasible next positions 

      next_positions = self.__next_pos(node[0], node[1], grid)

      for next_position in next_positions:
        # Cost till current node 
        g = cumumlative_cost[node] + self.__cost(next_position[0], next_position[1], grid)
        # exploring the next node and adding it the queue only if it does not \
        # add more cost to the overall cumulative cost
        if ( ((next_position not in source)  and (not pq.contains(next_position))) or (g < cumumlative_cost[next_position]) ):
          source[next_position] = node
          cumumlative_cost[next_position] =  g
         # f = g + h         
          f = g + self.__heuristic(next_position[0], next_position[1], \
                                   goal_x, goal_y,\
                                   'manhattan')
          pq.push(next_position, f)


  def __trace(self, source):

    node = (self.__food_row, self.__food_col)

    traversal = [node]

    while not self.__is_start(node[0], node[1], self.__grid):
      node = source[node]
      traversal.append(node)

    reverse_the_path = traversal[::-1]

    return reverse_the_path

  def play(self):
    output = self.__astar()
    traversal_path = self.__trace(output) 
    return traversal_path

Complete Code used for submission in Hackerrank 

In [19]:
import heapq

class PriorityQueue:
  def __init__(self):
    self._queue = []
    self._index = 0

  def is_empty(self):
    return not self._queue

  def push(self, item, priority):
    heapq.heappush(self._queue, (priority, self._index, item))
    self._index += 1   

  def pop(self):
    return heapq.heappop(self._queue)[-1]

          
  def contains(self, item):
    return not self.is_empty() and item in list(zip(*self._queue))[1]

class Pacman:
  
  # Input that is given as the input on hackerrank problem for the exam
  def __init__(self, start_row, start_col, food_row, food_col, grid):
    self.__start_row =  start_row
    self.__start_col = start_col
    self.__food_row = food_row
    self.__food_col = food_col
    self.__grid = grid

  # Defing the heuristic with choice of choosing between mahattan or euclidean 
  def __heuristic(self, source_x, source_y,  dest_x, dest_y, type):
    if type =="manhattan":
      dist = abs(dest_x - source_x) + abs(dest_y - source_y)
    elif type == "euclidean":
      dist =  ((dest_x - source_x)**2 + (dest_y - source_y)**2)**(0.5)
    return dist

  # Finding out whether the goal node is reached or not 
  def __is_goal(self, pos_x, pos_y, grid):
    return grid[pos_x][pos_y]  == '.'

  # Finding out whether the goal node is reached or not 
  def __is_wall(self, pos_x, pos_y, grid):
    return grid[pos_x][pos_y]  == '%'    
  
  # Finding the start position based on the problem 
  def __is_start(self, pos_x, pos_y, grid):
    return grid[pos_x][pos_y]  == 'P'
  
  # Finding the cost(g) every move
  def __cost(self, pos_x, pos_y, grid):
    cost = 0 
    if self.__is_goal(pos_x, pos_y, grid):
      cost = 1
    return cost

  # Feasible next positions 
  def __next_pos(self, pos_x, pos_y, grid):
    all_poss_next_positions = [\
                               (pos_x + 1, pos_y ), \
                               (pos_x , pos_y + 1 ),\
                               (pos_x , pos_y - 1  ),\
                               (pos_x - 1 , pos_y )\
                               ]
    next_pos = []                           
    for val in all_poss_next_positions:
      if not self.__is_wall(val[0], val[1], grid):
        next_pos.append(val)
    return next_pos 

  # Implementing the search algo from the start to the end   
  def __astar(self):
    pq = PriorityQueue()
    source  = {}
    cumumlative_cost = {}    
    goal_x = self.__food_row
    goal_y = self.__food_col
    start_x = self.__start_row
    start_y = self.__start_col
    grid = self.__grid
    # Starting from the first node 
    source[(start_x, start_y)] = None
    pq.push((start_x, start_y), 0)
    cumumlative_cost[(start_x, start_y)] = 0
    
    while not pq.is_empty():
      
      node = pq.pop()

      if self.__is_goal(node[0], node[1], grid):
        return source 
      
      # get the feasible next positions 

      next_positions = self.__next_pos(node[0], node[1], grid)

      for next_position in next_positions:
        # Cost till current node 
        g = cumumlative_cost[node] + self.__cost(next_position[0], next_position[1], grid)
        # exploring the next node and adding it the queue only if it does not \
        # add more cost to the overall cumulative cost
        if ( ((next_position not in source)  and (not pq.contains(next_position))) or (g < cumumlative_cost[next_position]) ):
          source[next_position] = node
          cumumlative_cost[next_position] =  g
         # f = g + h         
          f = g + self.__heuristic(next_position[0], next_position[1], \
                                   goal_x, goal_y,\
                                   'manhattan')
          pq.push(next_position, f)


  def __trace(self, source):

    node = (self.__food_row, self.__food_col)

    traversal = [node]

    while not self.__is_start(node[0], node[1], self.__grid):
      node = source[node]
      traversal.append(node)

    reverse_the_path = traversal[::-1]

    return reverse_the_path

  def play(self):
    output = self.__astar()
    traversal_path = self.__trace(output) 
    return traversal_path



# Boiler Plate Code for Hackerrank submissions
start_row, start_col = [ int(i) for i in input().strip().split() ]
food_row, food_col = [ int(i) for i in input().strip().split() ]
r,c = [ int(i) for i in input().strip().split() ]

grid = []
for i in range(0, r):
    grid.append(input().strip())
    
    
pacman = Pacman(start_row, start_col, food_row, food_col, grid)
#pacman.play()
path = pacman.play()
print(len(path)-1)
for node in path:
    print(f"{node[0]} {node[1]}")

35 35
35 1
37 37
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%-------%-%-%-----------%---%-----%-%
%-%%%%%%%-%-%%%-%-%%%-%%%-%%%%%%%-%-%
%-------%-------%-%-----%-----%-%---%
%%%%%-%%%%%-%%%-%-%-%-%%%-%%%%%-%-%%%
%---%-%-%-%---%-%-%-%---%-%---%-%---%
%-%%%-%-%-%-%%%-%%%%%-%%%-%-%%%-%%%-%
%-------%-----%---%---%-----%-%-%---%
%%%-%%%%%%%%%-%%%%%%%-%%%-%%%-%-%-%-%
%-------------%-------%-%---%-----%-%
%-%-%%%%%-%-%%%-%-%-%%%-%-%%%-%%%-%-%
%-%-%-----%-%-%-%-%-----%---%-%-%-%-%
%-%-%-%%%%%%%-%-%%%%%%%%%-%%%-%-%%%-%
%-%-%-%-----%---%-----%-----%---%---%
%%%-%%%-%-%%%%%-%%%%%-%%%-%%%-%%%%%-%
%-----%-%-%-----%-%-----%-%---%-%-%-%
%-%-%-%-%-%%%-%%%-%%%-%%%-%-%-%-%-%-%
%-%-%-%-%-----------------%-%-%-----%
%%%-%%%%%%%-%-%-%%%%%-%%%-%-%%%-%%%%%
%-------%-%-%-%-----%---%-----%-%---%
%%%%%-%-%-%%%%%%%%%-%%%%%%%%%%%-%-%%%
%---%-%-----------%-%-----%---%-%---%
%-%%%-%%%%%-%%%%%%%%%-%%%%%-%-%-%%%-%
%-%---%------%--------%-----%-------%
%-%-%-%%%%%-%%%-%-%-%-%-%%%%%%%%%%%%%
%-%-%---%-----%-%-%-%-------%---%